In [15]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [16]:
import os
import pandas as pd
import numpy as np

In [17]:
df2 = pd.read_csv('../Data/ABIDE_df3.csv')

In [18]:
df2.head()

,Unnamed: 0,participant_id,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,...,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,bids_folder,site,funcFile
0,0,50002,1,1.0,16.77,1,Ambi,NaN,103.0,116.0,...,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050002,13,Pitt_0050002_func_preproc.nii.gz
1,1,50003,1,1.0,24.45,1,R,NaN,124.0,128.0,...,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050003,13,Pitt_0050003_func_preproc.nii.gz
2,2,50004,1,1.0,19.09,1,R,NaN,113.0,108.0,...,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050004,13,Pitt_0050004_func_preproc.nii.gz
3,3,50005,1,1.0,13.73,2,R,NaN,119.0,117.0,...,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050005,13,Pitt_0050005_func_preproc.nii.gz
4,4,50006,1,1.0,13.37,1,L,NaN,109.0,99.0,...,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050006,13,Pitt_0050006_func_preproc.nii.gz


In [19]:
subs = df2['participant_id'].values

In [20]:
# How many folders
sum([os.path.exists(f'../../data/ABIDE/Derivatives/fmriprep/fmriprep/sub-00{sub}/') for sub in subs])

1046

In [21]:
is_vec = list()
for sub in subs:
    subfolder = f'../../data/ABIDE/Derivatives/fmriprep/fmriprep/sub-00{sub}/func/'
    if os.path.isdir(subfolder):
        conts = os.listdir(subfolder)
        has_nii = any(['preproc_bold.nii.gz' in cont for cont in conts])
    else:
        has_nii = False
    is_vec.append(has_nii)
    
is_vec = np.array(is_vec)
print(f'{is_vec.sum()}/{len(is_vec)}')
subfolder

1026/1048


'../../data/ABIDE/Derivatives/fmriprep/fmriprep/sub-0051607/func/'

In [22]:
bids_root = '../../data/ABIDE/RawDataBIDS/'
bids_folders = os.listdir(bids_root)
bids_folders.sort()

In [23]:
bids_df = None
for f in range(len(bids_folders)):
    bids_folder = bids_folders[f]
    bids_subs = [bids_sub for bids_sub in os.listdir(os.path.join(bids_root,bids_folder)) if bids_sub.startswith('sub-')]
    bids_subs.sort()

    temp_df = pd.DataFrame(bids_subs,columns=['bids_id'])
    temp_df['siteID'] = bids_folder
    temp_df['participant_id'] = [int(line.replace('sub-00','')) for line in temp_df['bids_id'].values]

    if type(bids_df)==type(None):
        bids_df=temp_df
    else:
        bids_df = pd.concat((bids_df,temp_df))
        
bids_df.head(5)

,bids_id,siteID,participant_id
0,sub-0050642,CMU_a,50642
1,sub-0050646,CMU_a,50646
2,sub-0050647,CMU_a,50647
3,sub-0050649,CMU_a,50649
4,sub-0050653,CMU_a,50653


In [24]:
unique,counts=np.unique(bids_df['participant_id'].values,return_counts=True)
assert all(counts==1), 'not all unique'

In [25]:
# Add SiteID and BIDS ID to df2
df2['siteID'] = [bids_df.iloc[bids_df['participant_id'].values==df2['participant_id'].values[i]]['siteID'].values[0] for i in range(len(df2))]
df2['bids_id'] = [bids_df.iloc[bids_df['participant_id'].values==df2['participant_id'].values[i]]['bids_id'].values[0] for i in range(len(df2))]


In [26]:
# populate is_fmriprepped
fmriprepped_temp = '../../data/ABIDE/Derivatives/fmriprep/fmriprep/{sub}/func/{sub}_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
is_vec = [os.path.exists(fmriprepped_temp.format(sub=sub)) for sub in df2['bids_id'].values]
df2['is_fmriprepped'] = is_vec

In [27]:
print('{}/{}'.format(df2['is_fmriprepped'].values.sum(),len(df2)))

1026/1048


In [28]:
df_new = df2.groupby(['siteID']).mean()['is_fmriprepped']
df_new.sort_values(ascending=False)

siteID
OHSU        1.000000
Olin        1.000000
CMU_b       1.000000
Leuven_2    1.000000
MaxMun_a    1.000000
MaxMun_b    1.000000
MaxMun_c    1.000000
MaxMun_d    1.000000
NYU         1.000000
USM         1.000000
CMU_a       1.000000
Pitt        1.000000
SBL         1.000000
SDSU        1.000000
Stanford    1.000000
Trinity     1.000000
UCLA_1      1.000000
UCLA_2      1.000000
UM_1        1.000000
UM_2        1.000000
KKI         0.981818
Leuven_1    0.965517
Caltech     0.857143
Yale        0.714286
Name: is_fmriprepped, dtype: float64